### Initialize GEE


In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=AJHYg1WRIFns_724r5IxgYEzMVCt58yzm26aVX-xoZc&tc=SAqmppcba6BQK_odQXmWvtXZenJmcjw49fZi1VRlh04&cc=FQu831u3kBQMqfTtgGxX_qRDTRNtfnUVAj7XRb6_yn4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnEjAaBjAVJ5lk04GCLD63GU2cqvSk3fOLvZZFhvwCwCqSUHV1dasQ

Successfully saved authorization token.


### Input Parameters (customizable)

In [ ]:
GRES_ID = [127,34]

# Set buffer size for the input polygon to control the image processing extent
buffer_size = 250

# Specify the polygon data source
GRes_polys = ee.FeatureCollection('projects/ee-climatechangehydrology1/assets/GRes_Polys')

# Specify where to save the image composite as an GEE Asset
AssetDirectory = 'projects/ee-climatechangehydrology/assets/GRes_ProbabilityRaster/'

# Specify where to save the image composite as a GEOTIFF on Google Drive
Folder = 'GRes_Raster_Raw'

# Specify the dates for image collection
Yr2015 =  ee.Filter.date('2015-06-01', '2015-09-01')
Yr2016 =  ee.Filter.date('2016-06-01', '2016-09-01')
Yr2017 =  ee.Filter.date('2017-06-01', '2017-09-01')
Yr2018 =  ee.Filter.date('2018-06-01', '2018-09-01')
Yr2019 =  ee.Filter.date('2019-06-01', '2019-09-01')
Yr2020 =  ee.Filter.date('2020-06-01', '2020-09-01')
Yr2021 =  ee.Filter.date('2021-06-01', '2021-09-01')
Yr2022 =  ee.Filter.date('2022-06-01', '2022-09-01')
Yr2023 =  ee.Filter.date('2023-06-01', '2023-09-01')

# Combine these dates into a single time filter
TimeFilter = ee.Filter.Or(
    Yr2015, Yr2016, Yr2017,Yr2018,Yr2019,Yr2020,Yr2021,Yr2022
)

# For very large reservoirs, export them directly as raster for processing
# on ArcGIS Pro, use "GeoTIFF"

# Whether to export the result as GEE Asset (ee.Image) or GeoTIFF?
############################################################################
# ********* Enter your options: 'Asset' or 'GeoTIFF' ***********************
############################################################################
# Note: large image composite might have issues exporting as GEE Asset,
#       if that happens, try export as GeoTIFF, then manually upload to GEE
#       as an asset.

ExportFormat = 'GeoTIFF'

### Initiate the Batch Export Process

In [ ]:
for id in GRES_ID:
  try:
    # A polygon input is needed to set image process extent
    aoi = GRes_polys.filter(ee.Filter.eq('GRES_ID', id))

    # Create a buffer on the input polygon capture more areas beyond what's given
    aoi_buff = (aoi.geometry()).buffer(buffer_size)
    aoi_buff = ee.Feature(aoi_buff)

    # Set up a function to clip the selected images within the buffer
    clip_to_aoi_buff = lambda imagecollection: imagecollection.clip(aoi_buff)

    # Create an image composite from GEE Dataset "Dynamic World"
    # based on the given dates and geometry;
    # Merge the image collection into an ee.Image using the .mean() aggregation function;
    # .mean() enables filtering noises and misclassifications in later steps;
    # .mean() is better than .median() because we want the extreme values to stand out;
    # The quality of the image composite improves with the number of images acquired (i.e. wider date range);

    raw_prob_raster = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
      .filter(TimeFilter)
      .filterBounds(aoi)
      .map(clip_to_aoi_buff)
      .mean()
      .select('water')
      .rename('probability')
    )

    # Prep the export tasks
    export_to_asset = ee.batch.Export.image.toAsset(
      image = raw_prob_raster,
      description = 'GRES_' + str(id) + '_Image',
      assetId = AssetDirectory + 'GRES_' + str(id),
      region = aoi_buff.geometry(),
      scale=10,
      maxPixels=1e13
    )

    export_to_drive = ee.batch.Export.image.toDrive(
        image = raw_prob_raster,
        region = aoi_buff.geometry(),
        folder=Folder,
        description= 'GRES_' + str(id) + '_Image',
        fileFormat='GeoTIFF',
        scale=10, maxPixels=1e13
    )

    if ExportFormat == "Asset":
      # Start the task
      export_to_asset.start()

    elif ExportFormat == 'GeoTIFF':
      # Start the task
      export_to_drive.start()

    print(f"Operation Successful on GRES_ID: {id}.")

  except Exception as e:
    print(f"Operation Failed on GRES_ID: {id}. Please check your input and parameters.")


print ("If the operation is successful, the task will be submitted to GEE's Task Manager")

Operation Successful on GRES_ID: 127.
Operation Successful on GRES_ID: 34.
If the operation is successful, the task will be submitted to GEE's Task Manager
